# Econometrics Assignment 10
## Nicolas Wesseler

### Activate Environment

In [12]:
using Pkg
Pkg.activate(joinpath(pwd(),".."))
using Random, Distributions, PrettyTables, SCS, Plots, LinearAlgebra, Convex
using Base.Threads: @threads, @spawn
Random.seed!(1234)

  Activating project at `~/Desktop/MachineLearning`


TaskLocalRNG()

### Install missing packages

### Load Packages

### Ensure Replicability of Simulations

In [13]:
const k = 50;
const α = 1;
const n = 300;

## Generate the data

## a)

In [14]:
function dgp(ρ, σ, β1, β2)

    # create X
    means = zeros(50)
    cov  = fill(ρ, (k,k))
    cov[diagind(cov)] .= 1
    normal = MvNormal(means,cov)
    X = rand(normal,n)'

    # create Y 
    U = rand(Normal(0, σ^2), n)
    Y = α .+ β1 * X[:,1] .+ β2 * X[:,2] .+ U
    
    return (Y = Y, X = X)
end

dgp (generic function with 1 method)

In [97]:
Y,X = dgp(0.2,0.1,0.1,0.1);

## b) 

In [16]:
MOI=Convex.MOI
opt = MOI.OptimizerWithAttributes(SCS.Optimizer, "eps_abs"=>1.0e-08,"eps_rel"=>1.0e-08, MOI.Silent() => true)

function Lasso(Y, X, λ)

    #define variables to optimize over
    N,K = size(X) 
    X=[X ones(N,1)]
    
    b = Variable(K+1) 
    
    Q=X'X/N;
    c=X'Y/N;
    
    L1=quadform(b, Q)/2  
    L2 = dot(c, b) 
    L3 = norm(b[1:K], 1)

    Sol=minimize(L1-L2+λ*L3)
    
    solve!(Sol, opt, verbose=false)
    
    estr=round.(vec(evaluate(b)),digits=5)

    return estr
end

Lasso (generic function with 2 methods)

## TEST THE SPEED OF ONE ITERATION

In [31]:
Y,X = dgp(0.2,2.0,2.0,2.0)
lambda =  1.1 * 2.0 * sqrt((2 * log(n * k)) / n)
@time Lasso(Y, X, lambda)
test = 0
test += b_i[1] != 0 && b_i[2] != 0 && sum((b_i[3:50] .!= 0) ) > 0 ? 1.0 : 0.0

  0.239240 seconds (4.65 k allocations: 1021.234 KiB)


1.0

## c)

In [217]:
function simulation(ρ, σ, β1, β2)

    Y,X = dgp(ρ,σ,β1,β2)

    λ = 1.1 * σ * sqrt((2 * log(n * k)) / n)

    b_i = Lasso(Y, X, λ)
    
    count_1 = 0
    count_1 += b_i[1] != 0 ? 1.0 : 0.0

    count_2 = 0
    count_2 += b_i[2] != 0 ? 1.0 : 0.0

    count_3 = 0
    count_3 += b_i[1] != 0 && b_i[2] != 0 && sum((b_i[3:50] .!= 0) ) > 0 ? 1.0 : 0.0

    return (count_1 = count_1, count_2 = count_2, count_3 = count_3)
end

simulation (generic function with 1 method)

In [218]:
sto_1_c = 0
sto_2_c = 0
sto_3_c = 0

R = 10000

for i = 1:R

    @unpack count_1, count_2, count_3 = simulation(0.2,2.0,2.0,2.0)
    sto_1_c += count_1
    sto_2_c += count_2
    sto_3_c += count_3

end

sto_1_c = sto_1_c / R
sto_2_c = sto_2_c / R
sto_3_c = sto_3_c / R

0.7682

## d)

In [219]:
sto_1_d = 0
sto_2_d = 0
sto_3_d = 0

R = 10000

for i = 1:R

    count_1, count_2, count_3 = simulation(0.2,0.1,0.1,0.1)
    sto_1_d += count_1
    sto_2_d += count_2
    sto_3_d += count_3

end

sto_1_d = sto_1_d / R
sto_2_d = sto_2_d / R
sto_3_d = sto_3_d / R

0.0

## e)

In [220]:
sto_1_e = 0
sto_2_e = 0
sto_3_e = 0

R = 10000

for i = 1:R

    count_1, count_2, count_3 = simulation(0.2,2.0,2.0,0.2)
    sto_1_e += count_1
    sto_2_e += count_2
    sto_3_e += count_3

end

sto_1_e = sto_1_e / R
sto_2_e = sto_2_e / R
sto_3_e = sto_3_e / R

0.0832

## f)

In [221]:
sto_1_f = 0
sto_2_f = 0
sto_3_f = 0

R = 10000

for i = 1:R

    count_1, count_2, count_3 = simulation(0.9,2.0,2.0,2.0)
    sto_1_f += count_1
    sto_2_f += count_2
    sto_3_f += count_3

end

sto_1_f = sto_1_f / R
sto_2_f = sto_2_f / R
sto_3_f = sto_3_f / R

0.8561

In [222]:
table_header = ["Specification","Probability 1", "Probability 2", "Probability 3"]
content = ["c)" sto_1_c sto_2_c sto_3_c ; "d)" sto_1_d sto_2_d sto_3_d ; "e)" sto_1_e sto_2_e sto_3_e ; "f)" sto_1_f sto_2_f sto_3_f]
pretty_table(content, table_header)

┌───────────────┬───────────────┬───────────────┬───────────────┐
│ Specification │ Probability 1 │ Probability 2 │ Probability 3 │
├───────────────┼───────────────┼───────────────┼───────────────┤
│            c) │           1.0 │           1.0 │        0.7682 │
│            d) │           1.0 │           1.0 │           0.0 │
│            e) │           1.0 │        0.1132 │        0.0832 │
│            f) │         0.931 │        0.9347 │        0.8561 │
└───────────────┴───────────────┴───────────────┴───────────────┘
